# Imports

In [2]:
import sklearn
import pandas as pd
import numpy as np
import re

# Cleaning Data

In [3]:
F17_df = pd.read_csv('./Dataset/FIFA17_official_data.csv')
F18_df = pd.read_csv('./Dataset/FIFA18_official_data.csv')
F19_df = pd.read_csv('./Dataset/FIFA19_official_data.csv')
F20_df = pd.read_csv('./Dataset/FIFA20_official_data.csv')
F21_df = pd.read_csv('./Dataset/FIFA21_official_data.csv')
F22_df = pd.read_csv('./Dataset/FIFA22_official_data.csv')
datalist = [F17_df, F18_df, F19_df, F20_df, F21_df, F22_df]

In [4]:
# print(set(F18_df.columns) - set(F17_df.columns))
# print(set(F19_df.columns) - set(F18_df.columns))
# print(set(F20_df.columns) - set(F19_df.columns))
# print(set(F21_df.columns) - set(F20_df.columns))
# print(set(F22_df.columns) - set(F21_df.columns))
curr_set = F17_df
for next_set in datalist[1:]:
    print(set(next_set) - set(curr_set))
    curr_set = next_set

for dataset in datalist:
    if 'Release Clause' in list(dataset.columns):
        dataset.drop("Release Clause", axis = 1, inplace = True)
    if 'DefensiveAwareness' in list(dataset.columns):
        dataset.drop("DefensiveAwareness", axis = 1, inplace = True)
print("--------------------")
curr_set = F17_df
for next_set in datalist[1:]:
    print(set(next_set) - set(curr_set))
    curr_set = next_set

{'Release Clause'}
set()
{'DefensiveAwareness'}
set()
set()
--------------------
set()
set()
set()
set()
set()


In [5]:
#Now we can join the datasets\
F_all_df = pd.concat(datalist, ignore_index=True)
F_all_df.columns

Index(['ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag', 'Overall',
       'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight', 'Crossing', 'Finishing', 'HeadingAccuracy',
       'ShortPassing', 'Volleys', 'Dribbling', 'Curve', 'FKAccuracy',
       'LongPassing', 'BallControl', 'Acceleration', 'SprintSpeed', 'Agility',
       'Reactions', 'Balance', 'ShotPower', 'Jumping', 'Stamina', 'Strength',
       'LongShots', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Marking', 'StandingTackle', 'SlidingTackle',
       'GKDiving', 'GKHandling', 'GKKicking', 'GKPositioning', 'GKReflexes',
       'Best Position', 'Best Overall Rating'],
      dtype='object')

In [6]:
Unimportant_features  = ["Photo", "Flag", "Club Logo", "Wage", "Special", "International Reputation", "Work Rate", "Body Type", "Real Face", "Jersey Number", "Contract Valid Until", "Best Overall Rating"]
F_all_df.drop(Unimportant_features, axis = 1, inplace = True)
F_all_df["Ovr_pot"] = (F_all_df["Overall"] + F_all_df["Potential"])/2

In [7]:
# One-hot encode Preferred_foot
F_all_df = pd.get_dummies(F_all_df, columns = ["Preferred Foot"])
F_all_df

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Weak Foot,Skill Moves,...,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Ovr_pot,Preferred Foot_Left,Preferred Foot_Right
0,176580,L. Suárez,29,Uruguay,92,92,FC Barcelona,€83M,4.0,4.0,...,38.0,27.0,25.0,31.0,33.0,37.0,ST,92.0,0,1
1,178518,R. Nainggolan,28,Belgium,86,86,Roma,€37.5M,3.0,3.0,...,88.0,11.0,11.0,14.0,8.0,11.0,CDM,86.0,0,1
2,181872,A. Vidal,29,Chile,87,87,FC Bayern München,€41.5M,4.0,3.0,...,84.0,4.0,2.0,4.0,2.0,4.0,CDM,87.0,0,1
3,197445,D. Alaba,24,Austria,86,89,FC Bayern München,€41.5M,4.0,3.0,...,83.0,5.0,7.0,14.0,15.0,9.0,LB,87.5,1,0
4,195864,P. Pogba,23,France,88,94,Manchester United,€71.5M,4.0,5.0,...,73.0,5.0,6.0,2.0,4.0,3.0,CAM,91.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104347,240558,18 L. Clayton,17,England,53,70,Cheltenham Town,€100K,2.0,1.0,...,12.0,55.0,54.0,52.0,50.0,59.0,GK,61.5,0,1
104348,262846,�. Dobre,20,Romania,53,63,FC Academica Clinceni,€180K,2.0,1.0,...,12.0,57.0,52.0,53.0,48.0,58.0,GK,58.0,0,1
104349,241317,21 Xue Qinghao,19,China PR,47,60,Shanghai Shenhua FC,€100K,2.0,1.0,...,9.0,49.0,48.0,45.0,38.0,52.0,GK,53.5,0,1
104350,259646,A. Shaikh,18,India,47,67,ATK Mohun Bagan FC,€110K,3.0,1.0,...,13.0,49.0,41.0,39.0,45.0,49.0,GK,57.0,0,1


In [8]:
detailed_features = ['Age','Overall', 'Potential', 'Weak Foot', 'Skill Moves', 'Height', 'Weight', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes', "Ovr_pot", 'Preferred Foot_Left', 'Preferred Foot_Right']

In [9]:
F_all_df[detailed_features] = F_all_df[detailed_features].fillna(0)
F_all_df

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Weak Foot,Skill Moves,...,SlidingTackle,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Ovr_pot,Preferred Foot_Left,Preferred Foot_Right
0,176580,L. Suárez,29,Uruguay,92,92,FC Barcelona,€83M,4.0,4.0,...,38.0,27.0,25.0,31.0,33.0,37.0,ST,92.0,0,1
1,178518,R. Nainggolan,28,Belgium,86,86,Roma,€37.5M,3.0,3.0,...,88.0,11.0,11.0,14.0,8.0,11.0,CDM,86.0,0,1
2,181872,A. Vidal,29,Chile,87,87,FC Bayern München,€41.5M,4.0,3.0,...,84.0,4.0,2.0,4.0,2.0,4.0,CDM,87.0,0,1
3,197445,D. Alaba,24,Austria,86,89,FC Bayern München,€41.5M,4.0,3.0,...,83.0,5.0,7.0,14.0,15.0,9.0,LB,87.5,1,0
4,195864,P. Pogba,23,France,88,94,Manchester United,€71.5M,4.0,5.0,...,73.0,5.0,6.0,2.0,4.0,3.0,CAM,91.0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104347,240558,18 L. Clayton,17,England,53,70,Cheltenham Town,€100K,2.0,1.0,...,12.0,55.0,54.0,52.0,50.0,59.0,GK,61.5,0,1
104348,262846,�. Dobre,20,Romania,53,63,FC Academica Clinceni,€180K,2.0,1.0,...,12.0,57.0,52.0,53.0,48.0,58.0,GK,58.0,0,1
104349,241317,21 Xue Qinghao,19,China PR,47,60,Shanghai Shenhua FC,€100K,2.0,1.0,...,9.0,49.0,48.0,45.0,38.0,52.0,GK,53.5,0,1
104350,259646,A. Shaikh,18,India,47,67,ATK Mohun Bagan FC,€110K,3.0,1.0,...,13.0,49.0,41.0,39.0,45.0,49.0,GK,57.0,0,1


In [10]:
# Clean height column
feet_inches_re = re.compile(r"(\d)'(\d+)")

# Define a function to convert feet'inches to cm
def feet_inches_to_cm(s):
    match = feet_inches_re.match(s)
    if match:
        feet = int(match.group(1))
        inches = int(match.group(2))
        return round((feet*12 + inches) * 2.54, 2)
    else:
        return float(s.replace('cm', ''))
F_all_df['Height'] = pd.to_numeric(F_all_df['Height'].apply(feet_inches_to_cm))

In [11]:
F_all_df['Height'].value_counts()

182.88    13593
177.80    12221
175.26    10661
180.34    10380
187.96     9448
          ...  
159.00        3
156.00        2
155.00        1
157.00        1
206.00        1
Name: Height, Length: 71, dtype: int64

In [12]:
F_all_df['Weight'].value_counts()

154lbs    7042
165lbs    6815
176lbs    4958
159lbs    4800
172lbs    4717
          ... 
104kg        1
51kg         1
53kg         1
106lbs       1
105kg        1
Name: Weight, Length: 117, dtype: int64

In [13]:
# Clean Weight Column
def convert_weight(weight_str):
    if 'kg' in weight_str:
        # If weight is in kg, remove the 'kg' suffix and return as float
        return float(weight_str.replace('kg', ''))
    elif 'lbs' in weight_str:
        # If weight is in lbs, convert to kg and return as float
        return float(weight_str.replace('lbs', '')) * 0.453592
    else:
        print("oops")
        # Return NaN if weight format is not recognized
        return pd.np.nan

# Apply the function to the Weight column and create a new column 'Weight_kg'
F_all_df['Weight'] = F_all_df['Weight'].apply(convert_weight)

In [14]:
F_all_df['Weight'].value_counts()

69.853168     7042
74.842680     6815
79.832192     4958
72.121128     4800
78.017824     4717
              ... 
104.000000       1
51.000000        1
53.000000        1
48.080752        1
105.000000       1
Name: Weight, Length: 117, dtype: int64

In [15]:
F_all_df['Height'].value_counts()

182.88    13593
177.80    12221
175.26    10661
180.34    10380
187.96     9448
          ...  
159.00        3
156.00        2
155.00        1
157.00        1
206.00        1
Name: Height, Length: 71, dtype: int64

# Creating BMI to replace Height and Weight

In [16]:
F_all_df['Bmi'] = F_all_df['Weight'] / (F_all_df['Height']/100)**2

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = sklearn.preprocessing.StandardScaler()
F_norm_df = F_all_df.copy(deep = True)
F_norm_df[detailed_features] = scaler.fit_transform(F_norm_df[detailed_features].to_numpy())
F_norm_df

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Weak Foot,Skill Moves,...,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Ovr_pot,Preferred Foot_Left,Preferred Foot_Right,Bmi
0,176580,L. Suárez,0.873366,Uruguay,3.647373,3.353318,FC Barcelona,€83M,1.552616,2.090192,...,0.618657,0.538418,0.922738,1.007493,1.168766,ST,3.876224,-0.558918,0.558918,25.768347
1,178518,R. Nainggolan,0.665442,Belgium,2.786419,2.355094,Roma,€37.5M,0.053943,0.792811,...,-0.301205,-0.303018,-0.118727,-0.479603,-0.302032,CDM,2.855590,-0.558918,0.558918,21.117175
2,181872,A. Vidal,0.873366,Chile,2.929911,2.521465,FC Bayern München,€41.5M,1.552616,0.792811,...,-0.703645,-0.843941,-0.731353,-0.836506,-0.698015,CDM,3.025696,-0.558918,0.558918,23.012574
3,197445,D. Alaba,-0.166257,Austria,2.786419,2.854206,FC Bayern München,€41.5M,1.552616,0.792811,...,-0.646154,-0.543428,-0.118727,-0.063216,-0.415170,LB,3.110749,1.789172,-1.789172,23.430985
4,195864,P. Pogba,-0.374181,France,3.073404,3.686059,Manchester United,€71.5M,1.552616,3.387573,...,-0.646154,-0.603530,-0.853879,-0.717538,-0.754585,CAM,3.706119,-0.558918,0.558918,23.123158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104347,240558,18 L. Clayton,-1.621729,England,-1.948825,-0.306835,Cheltenham Town,€100K,-1.444731,-1.801951,...,2.228417,2.281393,2.209254,2.018719,2.413286,GK,-1.311998,-0.558918,0.558918,22.634676
104348,262846,�. Dobre,-0.997955,Romania,-1.948825,-1.471429,FC Academica Clinceni,€180K,-1.444731,-1.801951,...,2.343399,2.161187,2.270516,1.899751,2.356717,GK,-1.907368,-0.558918,0.558918,22.850462
104349,241317,21 Xue Qinghao,-1.205880,China PR,-2.809779,-1.970541,Shanghai Shenhua FC,€100K,-1.444731,-1.801951,...,1.883468,1.920777,1.780415,1.304912,2.017302,GK,-2.672843,-0.558918,0.558918,21.913580
104350,259646,A. Shaikh,-1.413805,India,-2.809779,-0.805947,ATK Mohun Bagan FC,€110K,0.053943,-1.801951,...,1.883468,1.500059,1.412839,1.721299,1.847595,GK,-2.077473,-0.558918,0.558918,21.389756


In [19]:
F_norm_df.columns

Index(['ID', 'Name', 'Age', 'Nationality', 'Overall', 'Potential', 'Club',
       'Value', 'Weak Foot', 'Skill Moves', 'Position', 'Joined',
       'Loaned From', 'Height', 'Weight', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes', 'Best Position', 'Ovr_pot',
       'Preferred Foot_Left', 'Preferred Foot_Right', 'Bmi'],
      dtype='object')

In [21]:
F_norm_df["Best Position"].value_counts()

CB     20687
ST     15332
CAM    12517
GK     10890
RM      8359
CDM     7824
CM      6018
RB      5116
LB      4840
LM      4711
RW      2656
RWB     1861
LWB     1779
LW      1279
CF       480
LF         2
SW         1
Name: Best Position, dtype: int64

In [22]:
F_norm_df

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Weak Foot,Skill Moves,...,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Ovr_pot,Preferred Foot_Left,Preferred Foot_Right,Bmi
0,176580,L. Suárez,0.873366,Uruguay,3.647373,3.353318,FC Barcelona,€83M,1.552616,2.090192,...,0.618657,0.538418,0.922738,1.007493,1.168766,ST,3.876224,-0.558918,0.558918,25.768347
1,178518,R. Nainggolan,0.665442,Belgium,2.786419,2.355094,Roma,€37.5M,0.053943,0.792811,...,-0.301205,-0.303018,-0.118727,-0.479603,-0.302032,CDM,2.855590,-0.558918,0.558918,21.117175
2,181872,A. Vidal,0.873366,Chile,2.929911,2.521465,FC Bayern München,€41.5M,1.552616,0.792811,...,-0.703645,-0.843941,-0.731353,-0.836506,-0.698015,CDM,3.025696,-0.558918,0.558918,23.012574
3,197445,D. Alaba,-0.166257,Austria,2.786419,2.854206,FC Bayern München,€41.5M,1.552616,0.792811,...,-0.646154,-0.543428,-0.118727,-0.063216,-0.415170,LB,3.110749,1.789172,-1.789172,23.430985
4,195864,P. Pogba,-0.374181,France,3.073404,3.686059,Manchester United,€71.5M,1.552616,3.387573,...,-0.646154,-0.603530,-0.853879,-0.717538,-0.754585,CAM,3.706119,-0.558918,0.558918,23.123158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104347,240558,18 L. Clayton,-1.621729,England,-1.948825,-0.306835,Cheltenham Town,€100K,-1.444731,-1.801951,...,2.228417,2.281393,2.209254,2.018719,2.413286,GK,-1.311998,-0.558918,0.558918,22.634676
104348,262846,�. Dobre,-0.997955,Romania,-1.948825,-1.471429,FC Academica Clinceni,€180K,-1.444731,-1.801951,...,2.343399,2.161187,2.270516,1.899751,2.356717,GK,-1.907368,-0.558918,0.558918,22.850462
104349,241317,21 Xue Qinghao,-1.205880,China PR,-2.809779,-1.970541,Shanghai Shenhua FC,€100K,-1.444731,-1.801951,...,1.883468,1.920777,1.780415,1.304912,2.017302,GK,-2.672843,-0.558918,0.558918,21.913580
104350,259646,A. Shaikh,-1.413805,India,-2.809779,-0.805947,ATK Mohun Bagan FC,€110K,0.053943,-1.801951,...,1.883468,1.500059,1.412839,1.721299,1.847595,GK,-2.077473,-0.558918,0.558918,21.389756


# Strikers

In [23]:
striker_df = F_norm_df[(F_norm_df["Best Position"] == "ST") | (F_norm_df["Best Position"] == "CF") | (F_norm_df["Best Position"] == "LF")]
striker_df  =  striker_df.reset_index(drop = True)
striker_df

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Weak Foot,Skill Moves,...,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Ovr_pot,Preferred Foot_Left,Preferred Foot_Right,Bmi
0,176580,L. Suárez,0.873366,Uruguay,3.647373,3.353318,FC Barcelona,€83M,1.552616,2.090192,...,0.618657,0.538418,0.922738,1.007493,1.168766,ST,3.876224,-0.558918,0.558918,25.768347
1,173731,G. Bale,0.249592,Wales,3.360388,3.020577,Real Madrid,€72M,0.053943,2.090192,...,-0.071239,-0.062607,-0.302515,-0.658054,-0.584877,ST,3.536013,1.789172,-1.789172,22.106529
2,20801,Cristiano Ronaldo,1.289216,Portugal,3.934357,3.686059,Real Madrid,€87M,1.552616,3.387573,...,-0.531171,-0.303018,-0.057464,-0.122700,-0.302032,ST,4.216436,-0.558918,0.558918,23.220144
3,184941,A. Sánchez,0.457517,Chile,3.073404,2.687836,Arsenal,€55M,0.053943,2.090192,...,-0.358697,-0.363120,-0.057464,-0.241667,-0.188893,ST,3.195802,-0.558918,0.558918,21.456996
4,54050,W. Rooney,1.081291,England,2.212450,1.689612,Manchester United,€18.5M,1.552616,0.792811,...,-0.358697,-0.303018,-0.179990,-0.479603,-0.528308,ST,2.175168,-0.558918,0.558918,27.024076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15809,262034,R. Mahon,-1.413805,Republic of Ireland,-2.666287,-1.970541,Bohemian FC,€110K,0.053943,-0.504570,...,-0.531171,-0.122710,-0.608828,-0.479603,-0.075755,ST,-2.587790,1.789172,-1.789172,20.549887
15810,259650,F. Molla,-1.205880,India,-2.809779,-0.805947,ATK Mohun Bagan FC,€130K,-1.444731,-0.504570,...,-0.473679,-0.603530,-0.118727,-0.301151,-0.132324,ST,-2.077473,-0.558918,0.558918,24.304618
15811,161659,07 Danilo,-0.166257,Brazil,-0.370411,-0.805947,Gallos Blancos de Querétaro,€0,0.053943,-1.801951,...,-0.301205,-0.603530,2.515567,-0.063216,-0.245462,ST,-0.631575,1.789172,-1.789172,24.930748
15812,150597,09 J. Moukoko,-0.374181,Congo DR,-1.805333,-1.637800,NaN,€0,0.053943,-1.801951,...,-0.473679,0.538418,1.474102,0.531623,0.489936,ST,-1.907368,-0.558918,0.558918,23.503556


In [24]:
striker_df.columns

Index(['ID', 'Name', 'Age', 'Nationality', 'Overall', 'Potential', 'Club',
       'Value', 'Weak Foot', 'Skill Moves', 'Position', 'Joined',
       'Loaned From', 'Height', 'Weight', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes', 'Best Position', 'Ovr_pot',
       'Preferred Foot_Left', 'Preferred Foot_Right', 'Bmi'],
      dtype='object')

In [25]:
ST_detailed_features = ['Weak Foot', 'Skill Moves', 'Height', 'Weight','Bmi', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle']
# Left and Right foot not used as this is central position.

In [26]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
X = striker_df[ST_detailed_features].to_numpy()
y = striker_df["Ovr_pot"].to_numpy()
# define feature selection
fs = SelectKBest(score_func=f_regression, k = 10)
# apply feature selection
X_selected = fs.fit_transform(X, y)

In [27]:
ST_essential_ft = list(fs.get_feature_names_out(ST_detailed_features))
ST_essential_ft

['Finishing',
 'HeadingAccuracy',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'BallControl',
 'Reactions',
 'ShotPower',
 'LongShots',
 'Positioning']

# Wingers

In [28]:
wingers_df = F_norm_df[(F_norm_df["Best Position"] == "LW") | (F_norm_df["Best Position"] == "RW")]
wingers_df  =  wingers_df.reset_index(drop = True)
wingers_df

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Weak Foot,Skill Moves,...,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Ovr_pot,Preferred Foot_Left,Preferred Foot_Right,Bmi
0,190871,Neymar,-0.166257,Brazil,3.647373,3.686059,FC Barcelona,€106M,3.051289,3.387573,...,-0.416188,-0.423223,-0.057464,-0.063216,-0.302032,LW,4.046330,-0.558918,0.558918,22.150882
1,193130,A. Schürrle,0.041668,Germany,2.068958,1.689612,Borussia Dortmund,€19M,1.552616,0.792811,...,-0.531171,-0.303018,-0.241252,-0.420119,-0.245462,RW,2.090115,-0.558918,0.558918,22.106529
2,209331,M. Salah,-0.166257,Egypt,2.212450,2.188724,Roma,€26M,-1.444731,2.090192,...,-0.128731,-0.122710,-0.425040,-0.301151,-0.132324,RW,2.430326,1.789172,-1.789172,23.479935
3,222492,L. Sané,-0.997955,Germany,2.212450,3.186947,Manchester City,€31M,0.053943,0.792811,...,-0.473679,-0.242915,-0.425040,-0.420119,-0.132324,LW,2.940643,1.789172,-1.789172,22.784644
4,185020,José Callejón,0.873366,Spain,2.355942,1.855983,Napoli,€22.5M,-1.444731,0.792811,...,-0.473679,-0.663633,-0.118727,-0.301151,-0.302032,RW,2.345273,-0.558918,0.558918,23.100839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3930,248281,20 D. Borrero,-1.621729,Colombia,-1.518349,0.525018,Club Independiente Santa Fe,€180K,0.053943,0.792811,...,-0.243714,-0.423223,-0.425040,-0.658054,-0.188893,RW,-0.631575,-0.558918,0.558918,21.847009
3931,263090,J. Eng,-1.829654,Norway,-1.948825,0.691388,Vålerenga Fotball,€300K,0.053943,0.792811,...,-0.358697,-0.363120,-0.547565,-0.360635,-0.641446,RW,-0.801681,-0.558918,0.558918,24.074074
3932,264675,J. Dickman,-0.997955,Sweden,-2.092318,-0.805947,FC Nordsjælland,€190K,0.053943,-0.504570,...,-0.128731,-0.062607,-0.241252,-0.539087,-0.584877,RW,-1.652209,-0.558918,0.558918,22.582709
3933,261677,R. O'Kane,-1.621729,Republic of Ireland,-2.235810,0.525018,Dundalk FC,€180K,-1.444731,-0.504570,...,-0.301205,-0.242915,-0.118727,-0.360635,-0.584877,RW,-1.056840,-0.558918,0.558918,20.904195


In [29]:
Wing_detailed_features = ['Weak Foot', 'Skill Moves', 'Height', 'Weight','Bmi', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle']

In [30]:
X = wingers_df[Wing_detailed_features].to_numpy()
y = wingers_df["Ovr_pot"].to_numpy()
# define feature selection
fs = SelectKBest(score_func=f_regression, k = 15)
# apply feature selection
X_selected = fs.fit_transform(X, y)

In [31]:
Wing_essential_ft = list(fs.get_feature_names_out(Wing_detailed_features))
Wing_essential_ft

['Skill Moves',
 'Crossing',
 'Finishing',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'Curve',
 'LongPassing',
 'BallControl',
 'Reactions',
 'ShotPower',
 'LongShots',
 'Positioning',
 'Vision',
 'Composure']

# Midfield

In [32]:
mids_df = F_norm_df[(F_norm_df["Best Position"] == "RM") | (F_norm_df["Best Position"] == "CM") | (F_norm_df["Best Position"] == "CDM") | (F_norm_df["Best Position"] == "LM")]
mids_df  =  mids_df.reset_index(drop = True)
mids_df

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Weak Foot,Skill Moves,...,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Ovr_pot,Preferred Foot_Left,Preferred Foot_Right,Bmi
0,178518,R. Nainggolan,0.665442,Belgium,2.786419,2.355094,Roma,€37.5M,0.053943,0.792811,...,-0.301205,-0.303018,-0.118727,-0.479603,-0.302032,CDM,2.855590,-0.558918,0.558918,21.117175
1,181872,A. Vidal,0.873366,Chile,2.929911,2.521465,FC Bayern München,€41.5M,1.552616,0.792811,...,-0.703645,-0.843941,-0.731353,-0.836506,-0.698015,CDM,3.025696,-0.558918,0.558918,23.012574
2,177003,L. Modrić,1.081291,Croatia,3.216896,2.854206,Real Madrid,€54M,1.552616,2.090192,...,-0.186222,-0.423223,-0.547565,-0.122700,-0.415170,CM,3.365907,-0.558918,0.558918,21.117175
3,173210,C. Marchisio,1.081291,Italy,2.786419,2.355094,Juventus,€35M,1.552616,2.090192,...,-0.703645,-0.783838,-0.731353,-0.836506,-0.698015,CM,2.855590,-0.558918,0.558918,22.454694
4,139997,D. Srna,1.912989,Croatia,2.068958,1.523241,Shakhtar Donetsk,€4.8M,0.053943,0.792811,...,-0.358697,-0.363120,-0.608828,-0.598570,-0.245462,CM,2.005062,-0.558918,0.558918,22.813872
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26907,4174,07 P. Sarr,0.665442,Senegal,-0.226918,-0.306835,Stade Brestois 29,€0,0.053943,-1.801951,...,-0.416188,-0.242915,3.189456,-0.479603,-0.302032,CDM,-0.291364,-0.558918,0.558918,23.456790
26908,246839,21 C. Bargary,-1.205880,Republic of Ireland,-2.235810,-0.805947,Cork City,€150K,0.053943,-0.504570,...,-0.358697,-0.603530,-0.302515,-0.598570,-0.641446,RM,-1.737262,-0.558918,0.558918,21.777553
26909,235352,18 T. Käßemodel,0.665442,Germany,-2.953271,-4.299729,FC Erzgebirge Aue,€30K,0.053943,-0.504570,...,-0.358697,-0.242915,-0.608828,-0.182184,-0.584877,CM,-3.948636,-0.558918,0.558918,25.059307
26910,4373,07 S. Valakari,1.705065,Finland,-0.083426,-0.805947,FC Dallas,€0,0.053943,-1.801951,...,-0.588662,-0.783838,2.270516,-0.598570,-0.754585,CDM,-0.461470,1.789172,-1.789172,26.753864


In [33]:
Mid_detailed_features = ['Weak Foot', 'Skill Moves', 'Height', 'Weight','Bmi', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle']

In [34]:
X = mids_df[Mid_detailed_features].to_numpy()
y = mids_df["Ovr_pot"].to_numpy()
# define feature selection
fs = SelectKBest(score_func=f_regression, k = 15)
# apply feature selection
X_selected = fs.fit_transform(X, y)

In [35]:
Mid_essential_ft = list(fs.get_feature_names_out(Mid_detailed_features))
Mid_essential_ft

['Crossing',
 'Finishing',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'Curve',
 'LongPassing',
 'BallControl',
 'Reactions',
 'ShotPower',
 'Stamina',
 'LongShots',
 'Positioning',
 'Vision',
 'Composure']

# CAM

In [36]:
cam_df = F_norm_df[(F_norm_df["Best Position"] == "CAM")]
cam_df  =  cam_df.reset_index(drop = True)
cam_df

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Weak Foot,Skill Moves,...,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Ovr_pot,Preferred Foot_Left,Preferred Foot_Right,Bmi
0,195864,P. Pogba,-0.374181,France,3.073404,3.686059,Manchester United,€71.5M,1.552616,3.387573,...,-0.646154,-0.603530,-0.853879,-0.717538,-0.754585,CAM,3.706119,-0.558918,0.558918,23.123158
1,20289,Y. Touré,1.705065,Ivory Coast,2.499435,2.022353,Manchester City,€18M,1.552616,0.792811,...,-0.186222,-0.122710,-0.608828,-0.241667,-0.471739,CAM,2.515379,-0.558918,0.558918,25.421414
2,189509,Thiago,0.041668,Spain,2.786419,2.687836,FC Bayern München,€46M,0.053943,3.387573,...,-0.588662,-0.303018,-0.547565,-0.420119,-0.188893,CAM,3.025696,-0.558918,0.558918,22.741573
3,171877,M. Hamšik,0.665442,Slovakia,2.786419,2.355094,Napoli,€37.5M,1.552616,0.792811,...,-0.473679,-0.603530,-0.731353,-0.122700,-0.132324,CAM,2.855590,-0.558918,0.558918,23.598381
4,180206,M. Pjanić,0.249592,Bosnia Herzegovina,2.642927,2.355094,Juventus,€37.5M,1.552616,2.090192,...,-0.531171,-0.543428,-0.179990,-0.539087,-0.471739,CAM,2.770538,-0.558918,0.558918,21.478402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12512,252881,21 H. Hopper,-1.205880,England,-2.379302,-0.805947,Rochdale,€120K,-1.444731,-0.504570,...,-0.473679,-0.543428,-0.670091,-0.479603,-0.358601,CAM,-1.822315,-0.558918,0.558918,21.877551
12513,243725,D. Collins,-0.997955,Republic of Ireland,-2.809779,-2.636023,Sligo Rovers,€100K,0.053943,-0.504570,...,-0.588662,-0.423223,-0.670091,-0.182184,-0.471739,CAM,-3.013055,-0.558918,0.558918,22.460034
12514,259857,A. Majok,-0.997955,South Sudan,-2.379302,-0.805947,Western United FC,€120K,-1.444731,-0.504570,...,-0.473679,-0.303018,-0.363777,-0.479603,-0.358601,CAM,-1.822315,-0.558918,0.558918,21.132713
12515,261748,C. McLaughlin,-1.205880,Republic of Ireland,-2.235810,-2.136912,Derry City,€130K,0.053943,0.792811,...,-0.243714,-0.423223,-0.302515,-0.122700,-0.528308,CAM,-2.417685,1.789172,-1.789172,22.857143


In [37]:
cam_df.columns

Index(['ID', 'Name', 'Age', 'Nationality', 'Overall', 'Potential', 'Club',
       'Value', 'Weak Foot', 'Skill Moves', 'Position', 'Joined',
       'Loaned From', 'Height', 'Weight', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes', 'Best Position', 'Ovr_pot',
       'Preferred Foot_Left', 'Preferred Foot_Right', 'Bmi'],
      dtype='object')

In [38]:
Cam_detailed_features = ['Weak Foot', 'Skill Moves', 'Height', 'Weight','Bmi', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle']

In [39]:
X = cam_df[Cam_detailed_features].to_numpy()
y = cam_df["Ovr_pot"].to_numpy()
# define feature selection
fs = SelectKBest(score_func=f_regression, k = 15)
# apply feature selection
X_selected = fs.fit_transform(X, y)

In [40]:
Cam_essential_ft = list(fs.get_feature_names_out(Cam_detailed_features))
Cam_essential_ft

['Skill Moves',
 'Crossing',
 'Finishing',
 'ShortPassing',
 'Volleys',
 'Dribbling',
 'Curve',
 'FKAccuracy',
 'LongPassing',
 'BallControl',
 'Reactions',
 'ShotPower',
 'LongShots',
 'Positioning',
 'Vision']

# Defenders

In [41]:
centrebacks_df = F_norm_df[(F_norm_df["Best Position"] == "CB")]
centrebacks_df  = centrebacks_df.reset_index(drop = True)
centrebacks_df

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Weak Foot,Skill Moves,...,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Ovr_pot,Preferred Foot_Left,Preferred Foot_Right,Bmi
0,185103,A. Kolarov,1.081291,Serbia,1.925466,1.356871,Manchester City,€10M,0.053943,-0.504570,...,-0.013748,-0.062607,-0.486303,-0.003732,-0.584877,CB,1.834957,1.789172,-1.789172,23.495549
1,155862,Sergio Ramos,1.081291,Spain,3.216896,2.854206,Real Madrid,€43.5M,0.053943,0.792811,...,-0.301205,-0.483325,-0.425040,-0.539087,-0.302032,CB,3.365907,-0.558918,0.558918,22.377775
2,164240,Thiago Silva,1.289216,Brazil,3.216896,2.854206,Paris Saint-Germain,€39.5M,0.053943,0.792811,...,-0.416188,-0.242915,-0.670091,-0.420119,-0.358601,CB,3.365907,-0.558918,0.558918,23.598381
3,179944,David Luiz,0.873366,Brazil,2.499435,2.022353,Chelsea,€22M,0.053943,0.792811,...,-0.301205,-0.242915,-0.363777,-0.539087,-0.132324,CB,2.515379,-0.558918,0.558918,23.752331
4,205600,S. Umtiti,-0.582106,France,2.355942,2.687836,FC Barcelona,€27M,0.053943,-0.504570,...,-0.071239,-0.363120,-0.118727,-0.241667,-0.075755,CB,2.770538,1.789172,-1.789172,22.377775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20682,164945,07 Nuno Mendes,0.665442,Portugal,0.634035,-0.306835,US Créteil-Lusitanos,€0,0.053943,-1.801951,...,-0.301205,-0.242915,0.616425,-0.182184,-0.188893,CB,0.218953,-0.558918,0.558918,24.075474
20683,252561,Zhang Yu,-1.413805,China PR,-2.379302,-0.473206,Hebei FC,€110K,-1.444731,-0.504570,...,-0.646154,-0.182813,-0.241252,-0.658054,-0.075755,CB,-1.652209,-0.558918,0.558918,21.132713
20684,262041,P. Maguire,-1.413805,Republic of Ireland,-2.522794,-0.805947,Sligo Rovers,€110K,0.053943,-0.504570,...,-0.186222,-0.663633,-0.057464,-0.539087,-0.584877,CB,-1.907368,-0.558918,0.558918,20.233553
20685,239427,21 A. Ajeti,-0.374181,Switzerland,-1.805333,-1.804170,FC St. Gallen 1879,€200K,0.053943,-0.504570,...,-0.128731,-0.182813,-0.363777,-0.420119,-0.358601,CB,-1.992421,1.789172,-1.789172,23.702162


In [42]:
Def_detailed_features = ['Weak Foot', 'Skill Moves', 'Height', 'Weight','Bmi', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle']

In [43]:
X = centrebacks_df[Def_detailed_features].to_numpy()
y = centrebacks_df["Ovr_pot"].to_numpy()
# define feature selection
fs = SelectKBest(score_func=f_regression, k = 15)
# apply feature selection
X_selected = fs.fit_transform(X, y)

In [44]:
Def_essential_ft = list(fs.get_feature_names_out(Def_detailed_features))
Def_essential_ft

['HeadingAccuracy',
 'ShortPassing',
 'Dribbling',
 'LongPassing',
 'BallControl',
 'Reactions',
 'ShotPower',
 'Strength',
 'LongShots',
 'Aggression',
 'Interceptions',
 'Vision',
 'Composure',
 'StandingTackle',
 'SlidingTackle']

# Full-Backs

In [45]:
fullbacks_df = F_norm_df[(F_norm_df["Best Position"] == "LB") | (F_norm_df["Best Position"] == "RB") | (F_norm_df["Best Position"] == "LWB") | (F_norm_df["Best Position"] == "RWB")]
fullbacks_df  = fullbacks_df.reset_index(drop = True)
fullbacks_df

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Weak Foot,Skill Moves,...,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Ovr_pot,Preferred Foot_Left,Preferred Foot_Right,Bmi
0,197445,D. Alaba,-0.166257,Austria,2.786419,2.854206,FC Bayern München,€41.5M,1.552616,0.792811,...,-0.646154,-0.543428,-0.118727,-0.063216,-0.415170,LB,3.110749,1.789172,-1.789172,23.430985
1,203551,A. Florenzi,0.041668,Italy,2.212450,2.188724,Roma,€20.5M,0.053943,0.792811,...,-0.416188,-0.483325,-0.363777,-0.301151,-0.358601,RB,2.430326,-0.558918,0.558918,22.503074
2,163631,L. Baines,1.289216,England,2.355942,1.855983,Everton,€14.5M,0.053943,0.792811,...,-0.473679,-0.062607,-0.363777,-0.182184,-0.245462,LWB,2.345273,1.789172,-1.789172,24.119543
3,189332,Jordi Alba,0.457517,Spain,2.786419,2.355094,FC Barcelona,€31.5M,0.053943,0.792811,...,-0.186222,-0.062607,-0.179990,-0.598570,-0.188893,LWB,2.855590,1.789172,-1.789172,23.493061
4,146530,Dani Alves,1.705065,Brazil,2.499435,2.022353,Juventus,€12.5M,0.053943,0.792811,...,-0.646154,-0.303018,-0.425040,-0.598570,-0.528308,RWB,2.515379,-0.558918,0.558918,22.807169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13591,248815,K. Berkoe,-1.205880,England,-1.948825,-1.138688,Salford City,€210K,0.053943,-0.504570,...,-0.416188,-0.543428,-0.547565,-0.479603,-0.188893,LB,-1.737262,1.789172,-1.789172,20.515086
13592,261549,B. Ganesan,-1.205880,India,-2.666287,-1.804170,Chennaiyin FC,€110K,0.053943,-0.504570,...,-0.128731,-0.303018,-0.179990,-0.598570,-0.584877,RB,-2.502738,-0.558918,0.558918,19.814053
13593,260767,Z. Bello,-1.621729,Australia,-2.379302,-0.306835,Melbourne Victory,€120K,0.053943,-0.504570,...,-0.531171,-0.423223,-0.547565,-0.479603,-0.415170,RB,-1.567157,-0.558918,0.558918,21.857278
13594,252466,21 D. Bouah,-1.413805,Italy,-2.522794,-0.805947,Roma,€110K,-1.444731,-0.504570,...,-0.128731,-0.663633,-0.302515,-0.182184,-0.302032,RB,-1.907368,-0.558918,0.558918,23.620288


In [46]:
Def_detailed_features = ['Weak Foot', 'Skill Moves', 'Height', 'Weight','Bmi', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle']

In [47]:
X = fullbacks_df[Def_detailed_features].to_numpy()
y = fullbacks_df["Ovr_pot"].to_numpy()
# define feature selection
fs = SelectKBest(score_func=f_regression, k = 15)
# apply feature selection
X_selected = fs.fit_transform(X, y)

In [48]:
Fullback_essential_ft = list(fs.get_feature_names_out(Def_detailed_features))
Fullback_essential_ft

['Crossing',
 'HeadingAccuracy',
 'ShortPassing',
 'Dribbling',
 'Curve',
 'LongPassing',
 'BallControl',
 'Reactions',
 'Aggression',
 'Interceptions',
 'Positioning',
 'Vision',
 'Composure',
 'StandingTackle',
 'SlidingTackle']

# Goal Keeper

In [49]:
gk_df = F_norm_df[(F_norm_df["Best Position"] == "GK")]
gk_df  = gk_df.reset_index(drop = True)
gk_df

,ID,Name,Age,Nationality,Overall,Potential,Club,Value,Weak Foot,Skill Moves,...,GKDiving,GKHandling,GKKicking,GKPositioning,GKReflexes,Best Position,Ovr_pot,Preferred Foot_Left,Preferred Foot_Right,Bmi
0,230167,Mateus Tangil,0.665442,Brazil,0.060066,-0.805947,Atlético Mineiro,€625K,0.053943,-1.801951,...,2.860822,3.122829,2.883143,2.851492,3.148685,GK,-0.376417,-0.558918,0.558918,24.009113
1,230318,Nelsildo Reis,0.665442,Brazil,1.351496,0.691388,Internacional,€4.6M,0.053943,-1.801951,...,3.435736,3.663752,3.740820,3.565298,3.261823,GK,1.154534,-0.558918,0.558918,24.248069
2,167495,M. Neuer,1.081291,Germany,3.647373,3.353318,FC Bayern München,€69.5M,1.552616,-1.801951,...,4.183124,4.445085,4.843547,4.457556,4.110360,GK,3.876224,-0.558918,0.558918,24.709662
3,230171,Carlos Travisso,1.497140,Brazil,-0.513903,-1.471429,Botafogo,€250K,0.053943,-1.801951,...,2.975805,1.620264,2.944405,2.851492,3.148685,GK,-1.056840,-0.558918,0.558918,23.012574
4,230322,Teixe Gaúcho,0.665442,Brazil,-1.374856,-2.469653,Ponte Preta,€100K,-1.444731,-1.801951,...,3.090788,2.341495,2.331779,1.483364,2.582993,GK,-2.077473,-0.558918,0.558918,23.238767
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10885,251433,B. Voll,-0.997955,Germany,-1.231364,-0.473206,F.C. Hansa Rostock,€375K,-1.444731,-1.801951,...,2.458382,2.642008,2.454304,2.316138,2.526424,GK,-0.971787,-0.558918,0.558918,22.353715
10886,240558,18 L. Clayton,-1.621729,England,-1.948825,-0.306835,Cheltenham Town,€100K,-1.444731,-1.801951,...,2.228417,2.281393,2.209254,2.018719,2.413286,GK,-1.311998,-0.558918,0.558918,22.634676
10887,262846,�. Dobre,-0.997955,Romania,-1.948825,-1.471429,FC Academica Clinceni,€180K,-1.444731,-1.801951,...,2.343399,2.161187,2.270516,1.899751,2.356717,GK,-1.907368,-0.558918,0.558918,22.850462
10888,241317,21 Xue Qinghao,-1.205880,China PR,-2.809779,-1.970541,Shanghai Shenhua FC,€100K,-1.444731,-1.801951,...,1.883468,1.920777,1.780415,1.304912,2.017302,GK,-2.672843,-0.558918,0.558918,21.913580


In [50]:
gk_df.columns

Index(['ID', 'Name', 'Age', 'Nationality', 'Overall', 'Potential', 'Club',
       'Value', 'Weak Foot', 'Skill Moves', 'Position', 'Joined',
       'Loaned From', 'Height', 'Weight', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes', 'Best Position', 'Ovr_pot',
       'Preferred Foot_Left', 'Preferred Foot_Right', 'Bmi'],
      dtype='object')

In [51]:
Gk_detailed_features = ['Height', 'Weight','Bmi', 'ShortPassing', 'Volleys', 'Curve','Agility', 'Reactions', 'Balance',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
        'Positioning', 'Vision', 'Composure','GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes']

In [52]:
X = gk_df[Gk_detailed_features].to_numpy()
y = gk_df["Ovr_pot"].to_numpy()
# define feature selection
fs = SelectKBest(score_func=f_regression, k = 10)
# apply feature selection
X_selected = fs.fit_transform(X, y)

In [53]:
Gk_essential_ft = list(fs.get_feature_names_out(Gk_detailed_features))
Gk_essential_ft

['Agility',
 'Reactions',
 'Jumping',
 'Vision',
 'Composure',
 'GKDiving',
 'GKHandling',
 'GKKicking',
 'GKPositioning',
 'GKReflexes']

# Finalizing all dataframes

In [54]:
gk_df=gk_df[Gk_essential_ft]
fullbacks_df = fullbacks_df[Fullback_essential_ft]
centrebacks_df = centrebacks_df[Def_essential_ft]
cam_df = cam_df[Cam_essential_ft]
mids_df = mids_df[Mid_essential_ft]
wingers_df = wingers_df[Wing_essential_ft]
striker_df= striker_df[ST_essential_ft]